In [1]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau

from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
X = pd.read_csv('train.csv')

In [3]:
Y = X[['label']]

In [4]:
X = X.drop(["label"], axis=1)

In [5]:
X_test = pd.read_csv('test.csv')

X_test = X_test.values.reshape(X_test.shape[0], 28, 28, 1)

In [6]:
X_train = X.values.reshape(X.shape[0], 28, 28, 1)

# Convert class vectors to binary class matrices
Y_train = tf.keras.utils.to_categorical(Y.values, 10)

In [7]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=42)

In [8]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

X_train /= 255
X_test /= 255
X_val /= 255

In [9]:
datagen = ImageDataGenerator(
        rotation_range=10,
        zoom_range = 0.10,
        width_shift_range=0.1, 
        height_shift_range=0.1)

datagen.fit(X_train)

In [10]:
#Build model
model = tf.keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                 padding='same',
                 input_shape=(28, 28, 1)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (5, 5), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(layers.Dropout(0.2))
  
model.add(layers.Flatten())  
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))
  
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))
  
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Dense(10, activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
batch_normalization_v1 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        25632     
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 24, 24, 32)        128       
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)    

In [11]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_acc',
                                factor=0.5,
                                patience=3,
                                min_lr=0.00001,
                                verbose=1)

In [12]:
model.fit(datagen.flow(X_train, Y_train, batch_size=128),
                    epochs=30,
                    validation_data=(X_val, Y_val),
                    verbose=1,
                    steps_per_epoch=X_train.shape[0] // 128,
                    callbacks=[reduce_lr])

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
296/296 [==============================] - 5637s 19s/step - loss: 0.5593 - acc: 0.8233 - val_loss: 9.9097 - val_acc: 0.1121
Epoch 2/30
296/296 [==============================] - 3728s 13s/step - loss: 0.1534 - acc: 0.9535 - val_loss: 0.1877 - val_acc: 0.9448
Epoch 3/30
296/296 [==============================] - 986s 3s/step - loss: 0.1015 - acc: 0.9688 - val_loss: 0.0407 - val_acc: 0.9886
Epoch 4/30
296/296 [==============================] - 1003s 3s/step - loss: 0.0811 - acc: 0.9756 - val_loss: 0.0278 - val_acc: 0.9926
Epoch 5/30
296/296 [==============================] - 999s 3s/step - loss: 0.0729 - acc: 0.9779 - val_loss: 0.0520 - val_acc: 0.9850
Epoch 6/30
296/296 [==============================] - 994s 3s/step - loss: 0.0608 - acc: 0.9816 - val_loss: 0.0477 - val_acc: 0.9857
Epoch 7/30
296/296 [==============================] - 1007s 3s/step - loss: 0.0560 - acc: 0.9830 - val_loss: 0.0207 - val_acc: 0.9936
Epoch 8/30
296/

In [13]:
Y_test = model.predict(X_test, batch_size=128, verbose=1)

28000/28000 [==============================] - 107s 4ms/sample


In [14]:
Y_test = np.argmax(Y_test, axis=1)

In [15]:
frame = pd.DataFrame({'Label': Y_test})
frame.insert(0, 'ImageId', range(1, 1 + len(frame)))
frame.to_csv('output.csv', index=False)